<a href="https://colab.research.google.com/github/piyush-an/INFO7374_Predict_StockPrice/blob/main/notebooks/2_FeatureMart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2: Using the features/factors you take and discovered, e.g., FRED, Fama-French website, ADS, AR, CAPM, momentum factors, volume, price/return lags, etc.) to construct a feature database
- The target variable Y can be either price or return
- Frequency could be either daily or monthly

## Install Dependency and files

In [1]:
%%bash

pip install pandas yfinance ta seaborn matplotlib pandas-datareader jinja2 fredapi openpyxl xgboost scikit-learn statsmodels mlflow

if [ ! -f "ads_index_most_current_vintage.xlsx" ]; then
    wget https://www.philadelphiafed.org/-/media/frbp/assets/surveys-and-data/ads/ads_index_most_current_vintage.xlsx
fi
if [ ! -f "F-F_Research_Data_Factors_daily.CSV" ]; then
  wget https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip
	unzip F-F_Research_Data_Factors_daily_CSV.zip
fi

Archive:  F-F_Research_Data_Factors_daily_CSV.zip
  inflating: F-F_Research_Data_Factors_daily.CSV  


--2023-12-12 22:01:21--  https://www.philadelphiafed.org/-/media/frbp/assets/surveys-and-data/ads/ads_index_most_current_vintage.xlsx
Resolving www.philadelphiafed.org (www.philadelphiafed.org)... 104.123.70.35, 104.123.70.19, 2600:1406:3c00::17d6:700a
Connecting to www.philadelphiafed.org (www.philadelphiafed.org)|104.123.70.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from www.philadelphiafed.org attempted to set domain to philadelphiafed-prod-cd.ase2.p.azure.frb.pvt
Cookie coming from www.philadelphiafed.org attempted to set domain to philadelphiafed-prod-cd.ase2.p.azure.frb.pvt
Length: 535381 (523K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘ads_index_most_current_vintage.xlsx’

     0K .......... .......... .......... .......... ..........  9%  839K 1s
    50K .......... .......... .......... .......... .......... 19% 1.51M 0s
   100K .......... .......... .......... .......... .......... 28% 4.88M 0s
   15

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import ta

In [3]:
# Define dataset start and end date => Two years worth of data
start_date = datetime(2021, 1, 1)
end_date = datetime(2023, 12, 31)

# Downloaded data
NVDA_STOCK = yf.download("NVDA", start_date, end_date)
NVDA_STOCK.describe()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
count,741.000000,741.000000,741.000000,741.000000,741.000000,7.410000e+02
mean,244.556666,249.227911,239.774919,244.723148,244.523119,4.602601e+07
std,108.705638,110.102043,106.879366,108.487984,108.533463,1.744612e+07
min,109.709999,117.349998,108.129997,112.269997,112.191505,1.462760e+07
25%,158.600006,162.142502,155.630005,159.100006,158.988754,3.456350e+07
50%,210.000000,214.330002,207.000000,210.750000,210.433960,4.489300e+07
75%,288.410004,292.200012,280.929993,289.100006,289.020874,5.548160e+07
max,502.160004,505.480011,492.220001,504.089996,504.045685,1.543911e+08


In [4]:
NVDA_STOCK["Returns"] = NVDA_STOCK["Adj Close"] - NVDA_STOCK["Adj Close"].shift(1)

In [5]:
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2021-01-04,131.042496,136.524994,129.625000,131.134995,130.840118,56064000,NaN
2021-01-05,130.997498,134.434998,130.869995,134.047501,133.746063,32276000,2.905945
2021-01-06,132.225006,132.449997,125.860001,126.144997,125.861328,58042400,-7.884735
2021-01-07,129.675003,133.777496,128.865005,133.440002,133.139908,46148000,7.278580
2021-01-08,133.625000,134.210007,130.419998,132.767502,132.468948,29252800,-0.670959
...,...,...,...,...,...,...,...
2023-12-06,472.149994,473.869995,454.119995,455.029999,455.029999,38059000,-10.630005
2023-12-07,457.000000,466.290009,456.040009,465.959991,465.959991,35082300,10.929993
2023-12-08,465.950012,477.410004,465.500000,475.059998,475.059998,35880300,9.100006


Calculating daily returns:

Produce the day's difference of the stock dataframe: (`np.log(nvda['Open']) - np.log(nvda['Open'].shift(+1))`)

In [6]:
# Daily return
NVDA_STOCK["Daily_Return"] = np.log(NVDA_STOCK["Adj Close"]) - np.log(
    NVDA_STOCK["Adj Close"].shift(1)
)
NVDA_STOCK = NVDA_STOCK.dropna()
NVDA_STOCK.head()

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return
Date,,,,,,,,
2021-01-05,130.997498,134.434998,130.869995,134.047501,133.746063,32276000,2.905945,0.021967
2021-01-06,132.225006,132.449997,125.860001,126.144997,125.861328,58042400,-7.884735,-0.060762
2021-01-07,129.675003,133.777496,128.865005,133.440002,133.139908,46148000,7.278580,0.056220
2021-01-08,133.625000,134.210007,130.419998,132.767502,132.468948,29252800,-0.670959,-0.005052
2021-01-11,134.187500,139.610001,133.625000,136.214996,135.908676,51834400,3.439728,0.025635


### Feature-set 1: Typical Price, Typical_Price_Return
- `Typical_Price` is the *mean* value of High, Low and Close values

In [7]:
NVDA_STOCK["Typical_Price"] = NVDA_STOCK[["High", "Low", "Close"]].mean(axis=1)
NVDA_STOCK["Typical_Price_Return"] = (
    np.log(NVDA_STOCK.Typical_Price) - np.log(NVDA_STOCK.Typical_Price.shift(+1))
) * 100.0
NVDA_STOCK = NVDA_STOCK.dropna()
NVDA_STOCK.head()

<ipython-input-7-eddaa5efae65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NVDA_STOCK["Typical_Price"] = NVDA_STOCK[["High", "Low", "Close"]].mean(axis=1)
<ipython-input-7-eddaa5efae65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NVDA_STOCK["Typical_Price_Return"] = (


,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return
Date,,,,,,,,,,
2021-01-06,132.225006,132.449997,125.860001,126.144997,125.861328,58042400,-7.884735,-0.060762,128.151665,-3.801774
2021-01-07,129.675003,133.777496,128.865005,133.440002,133.139908,46148000,7.278580,0.056220,132.027501,2.979580
2021-01-08,133.625000,134.210007,130.419998,132.767502,132.468948,29252800,-0.670959,-0.005052,132.465836,0.331452
2021-01-11,134.187500,139.610001,133.625000,136.214996,135.908676,51834400,3.439728,0.025635,136.483332,2.987773
2021-01-12,137.524994,137.544998,133.449997,134.847504,134.544266,29147600,-1.364410,-0.010090,135.280833,-0.884964


### Feature-set 2: Common Transforms
- `log of volume`
- `pct_change of volume`
- `difference in volume`
- `log of 5 day moving average of volume`
- `Daily volume vs. 200 day moving average`
- `Daily closing price vs. 50 day exponential moving average`

In [8]:
NVDA_STOCK["Volume_Log"] = np.log(NVDA_STOCK.Volume)
NVDA_STOCK["Volume_Differencing"] = NVDA_STOCK.Volume.diff()
NVDA_STOCK["Volume_Differencing_10"] = NVDA_STOCK.Volume.diff(10)
NVDA_STOCK["Volumne_Percent_Change"] = NVDA_STOCK.Volume.pct_change()

In [9]:
# Log of 5 day moving average of volume
NVDA_STOCK["MA_5"] = np.log(NVDA_STOCK.Volume.rolling(5).mean())

# Daily volume vs. 200 day moving average
NVDA_STOCK["Volumne_MA_200"] = (
    NVDA_STOCK.Volume / NVDA_STOCK.Volume.rolling(200).mean() - 1
)

# Daily closing price vs. 50 day Exponential Moving Avg
NVDA_STOCK["Close_EMA_50"] = NVDA_STOCK.Close / NVDA_STOCK.Close.ewm(span=50).mean() - 1

### Feature-set 3: Momentum Indicators

**1. AwesomeOscillatorIndicator**

In [10]:
NVDA_STOCK['Momentum_AwesomeOscillatorIndicator'] = ta.momentum.AwesomeOscillatorIndicator(NVDA_STOCK.High, NVDA_STOCK.Low,window1 = 5,window2 = 34, fillna=False).awesome_oscillator()

**2. Kaufman’s Adaptive Moving Average (KAMA)**

In [11]:
NVDA_STOCK['Momentum_KAMA'] = ta.momentum.KAMAIndicator(NVDA_STOCK.Close, fillna=False).kama()

**3. PercentagePriceOscillator**

In [12]:
NVDA_STOCK['Momentum_PercentagePVolumneOscillator'] = ta.momentum.PercentageVolumeOscillator(NVDA_STOCK.Volume, fillna=False).pvo()

**4. Rate of Change (ROC)**

In [13]:
NVDA_STOCK['Momentum_ROC'] = ta.momentum.ROCIndicator(NVDA_STOCK.Close, fillna=False).roc()

**5. Relative Strength Index (RSI)**

In [14]:
NVDA_STOCK['Momentum_RSI'] = ta.momentum.RSIIndicator(NVDA_STOCK.Close, fillna=False).rsi()

**6. Stochastic RSI**

In [15]:
NVDA_STOCK['Momentum_StochRSIIndicator'] = ta.momentum.StochRSIIndicator(NVDA_STOCK.Close, fillna=False).stochrsi()

**7. True strength index (TSI)**

In [16]:
NVDA_STOCK['Momentum_TSIIndicator'] = ta.momentum.TSIIndicator(NVDA_STOCK.Close, fillna=False).tsi()

### Feature-set 4: Trend Indicators

**1. Average Directional Movement Index (ADX)**

In [17]:
NVDA_STOCK['Trend_ADX'] = ta.trend.ADXIndicator(NVDA_STOCK.High,NVDA_STOCK.Low, NVDA_STOCK.Close, window = 20,fillna=False).adx()

**2. Aroon Indicator**

In [18]:
NVDA_STOCK['Trend_AroonIndicator'] = ta.trend.AroonIndicator(NVDA_STOCK.Close, NVDA_STOCK.Low, window=20, fillna=False).aroon_indicator()

**3. Commodity Channel Index (CCI)**

In [19]:
NVDA_STOCK['Trend_CCI'] = ta.trend.CCIIndicator(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close, window = 20,fillna=False).cci()

**4. Detrended Price Oscillator (DPO)**

In [20]:
NVDA_STOCK['Trend_DPO'] = ta.trend.DPOIndicator(NVDA_STOCK.Close, window = 20, fillna=False).dpo()

**5. EMA - Exponential Moving Average**

In [21]:
NVDA_STOCK['Trend_EMA'] = ta.trend.EMAIndicator(NVDA_STOCK.Close, window = 20, fillna=False).ema_indicator()

**6. Moving Average Convergence Divergence (MACD)**

In [22]:
NVDA_STOCK['Trend_MACD'] = ta.trend.MACD(NVDA_STOCK.Close, fillna=False).macd()

**7. Mass Index (MI)**

In [23]:
NVDA_STOCK['Trend_MI'] = ta.trend.MassIndex(NVDA_STOCK.High, NVDA_STOCK.Low, fillna=False).mass_index()

### Feature-set 5: Volumne Indicator

**1. Chaikin Money Flow (CMF)**

In [24]:
NVDA_STOCK['Volumne_CMF'] = ta.volume.ChaikinMoneyFlowIndicator(NVDA_STOCK.High,NVDA_STOCK.Low,NVDA_STOCK.Close, NVDA_STOCK.Volume,window = 20,fillna=False).chaikin_money_flow()

**2. Ease of movement (EoM, EMV)**

In [25]:
NVDA_STOCK['Volumne_EOM'] = ta.volume.EaseOfMovementIndicator(NVDA_STOCK.High,NVDA_STOCK.Low, NVDA_STOCK.Volume,window = 20,fillna=False).ease_of_movement()

**3. Force Index (FI)**

In [26]:
NVDA_STOCK['Volumne_FI'] = ta.volume.ForceIndexIndicator(NVDA_STOCK.Close, NVDA_STOCK.Volume,window = 20,fillna=False).force_index()

**4. Money Flow Index (MFI)**

In [27]:
NVDA_STOCK['Volumne_MFI'] = ta.volume.money_flow_index(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close,NVDA_STOCK.Volume, window=20, fillna=False)

**5. Volume Weighted Average Price (VWAP)**

In [28]:
NVDA_STOCK['Volumne_VWAP'] = ta.volume.VolumeWeightedAveragePrice(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close,NVDA_STOCK.Volume, window=20, fillna=False).volume_weighted_average_price()

### Feature-set 6: Volatility Indicators

**1. Average True Range (ATR)**

In [29]:
NVDA_STOCK['Volatility_ATR'] = ta.volatility.AverageTrueRange(NVDA_STOCK.High, NVDA_STOCK.Low, NVDA_STOCK.Close, window=20, fillna=False).average_true_range()

**2. Bollinger Bands**

In [30]:
NVDA_STOCK['Volatility_BB'] = ta.volatility.BollingerBands(NVDA_STOCK.Close, window=20, fillna=False).bollinger_wband()

**3. Donchian Channel**

In [31]:
NVDA_STOCK['Volatility_DonchainChannel'] = ta.volatility.DonchianChannel(NVDA_STOCK.High, NVDA_STOCK.Low,NVDA_STOCK.Close, window=20, fillna=False).donchian_channel_wband()

**4. Ulcer Index**

In [32]:
NVDA_STOCK['Volatility_UlcerIndex'] = ta.volatility.UlcerIndex(NVDA_STOCK.Close, window=20, fillna=False).ulcer_index()

**5. Keltner channel (KC)**

In [33]:
NVDA_STOCK['Volatility_KeltnerChannel'] = ta.volatility.keltner_channel_hband(NVDA_STOCK.High, NVDA_STOCK.Low,NVDA_STOCK.Close, window=20, fillna=False)

In [34]:
NVDA_STOCK.dropna(inplace = True)

In [35]:
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volumne_CMF,Volumne_EOM,Volumne_FI,Volumne_MFI,Volumne_VWAP,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel,Volatility_UlcerIndex,Volatility_KeltnerChannel
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-20,223.050003,224.330002,219.820007,221.029999,220.698547,14627600,-1.867203,-0.008425,221.726669,-0.282228,...,0.084371,-0.153841,1.493985e+07,51.186251,211.241463,6.042212,14.172475,14.049514,7.094438,217.316335
2021-10-21,220.970001,227.110001,220.830002,226.919998,226.579727,18759000,5.881180,0.026299,224.953334,1.444757,...,0.078493,63.439291,2.403992e+07,50.503076,211.161915,6.054101,14.522044,14.871919,7.085631,217.404334
2021-10-22,228.229996,231.300003,225.610001,227.259995,226.919205,24938400,0.339478,0.001497,228.056666,1.370116,...,0.029243,102.330801,2.255792e+07,55.989117,211.654961,6.035896,15.391338,16.820798,7.046261,217.932500
2021-10-25,229.729996,233.550003,227.699997,231.660004,231.312622,23023500,4.393417,0.019176,230.970001,1.269370,...,0.024907,55.137153,3.005751e+07,61.545610,212.410164,6.048602,17.165327,17.816328,6.950452,218.739167
2021-10-26,239.889999,252.589996,239.240005,247.169998,246.799347,48589800,15.486725,0.064806,246.333333,6.439780,...,0.108564,420.090991,9.896894e+07,70.541920,216.199521,6.792671,21.599453,26.493693,6.626018,220.877833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-06,472.149994,473.869995,454.119995,455.029999,455.029999,38059000,-10.630005,-0.023092,461.006663,-0.097565,...,-0.000349,240.784014,-1.040439e+08,45.548497,480.826582,14.441806,11.610976,11.537946,4.794956,492.747171
2023-12-07,457.000000,466.290009,456.040009,465.959991,465.959991,35082300,10.929993,0.023736,462.763336,0.380327,...,0.023526,-82.683753,-5.761596e+07,45.564510,480.740935,14.282716,11.601405,11.537681,5.081454,492.713503
2023-12-08,465.950012,477.410004,465.500000,475.059998,475.059998,35880300,9.100006,0.019341,472.656667,2.115349,...,0.095896,341.563080,-2.103244e+07,44.419039,480.885374,14.164080,11.463363,11.531003,5.242081,492.546837


In [36]:
NVDA_STOCK.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 540 entries, 2021-10-20 to 2023-12-12
Data columns (total 41 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Open                                   540 non-null    float64
 1   High                                   540 non-null    float64
 2   Low                                    540 non-null    float64
 3   Close                                  540 non-null    float64
 4   Adj Close                              540 non-null    float64
 5   Volume                                 540 non-null    int64  
 6   Returns                                540 non-null    float64
 7   Daily_Return                           540 non-null    float64
 8   Typical_Price                          540 non-null    float64
 9   Typical_Price_Return                   540 non-null    float64
 10  Volume_Log                             540 non-null    

### Feature-set 7: Fama-French Indicators

In [37]:
df_fama = pd.read_csv("./F-F_Research_Data_Factors_daily.CSV", skiprows=3)
df_fama = df_fama.iloc[:-1]
df_fama.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
df_fama["Date"] = pd.to_datetime(df_fama["Date"])
df_fama = df_fama[(df_fama["Date"] >= "2020-10-31") & (df_fama["Date"] <= "2023-10-31")]
fama = df_fama.set_index("Date")

In [38]:
fama.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 754 entries, 2020-11-02 to 2023-10-31
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  754 non-null    float64
 1   SMB     754 non-null    float64
 2   HML     754 non-null    float64
 3   RF      754 non-null    float64
dtypes: float64(4)
memory usage: 29.5 KB


In [39]:
NVDA_STOCK = pd.concat([NVDA_STOCK, fama], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volumne_VWAP,Volatility_ATR,Volatility_BB,Volatility_DonchainChannel,Volatility_UlcerIndex,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-20,223.050003,224.330002,219.820007,221.029999,220.698547,14627600.0,-1.867203,-0.008425,221.726669,-0.282228,...,211.241463,6.042212,14.172475,14.049514,7.094438,217.316335,0.34,-0.02,1.19,0.000
2021-10-21,220.970001,227.110001,220.830002,226.919998,226.579727,18759000.0,5.881180,0.026299,224.953334,1.444757,...,211.161915,6.054101,14.522044,14.871919,7.085631,217.404334,0.37,0.21,-0.95,0.000
2021-10-22,228.229996,231.300003,225.610001,227.259995,226.919205,24938400.0,0.339478,0.001497,228.056666,1.370116,...,211.654961,6.035896,15.391338,16.820798,7.046261,217.932500,-0.25,-0.24,1.01,0.000
2021-10-25,229.729996,233.550003,227.699997,231.660004,231.312622,23023500.0,4.393417,0.019176,230.970001,1.269370,...,212.410164,6.048602,17.165327,17.816328,6.950452,218.739167,0.58,0.49,-0.15,0.000
2021-10-26,239.889999,252.589996,239.240005,247.169998,246.799347,48589800.0,15.486725,0.064806,246.333333,6.439780,...,216.199521,6.792671,21.599453,26.493693,6.626018,220.877833,0.04,-0.70,-0.31,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,440.555158,16.208317,14.665447,15.080417,7.275335,455.716169,-1.58,-0.30,0.84,0.021
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,439.196002,16.585902,16.474270,17.545335,7.403454,455.207336,-1.15,0.85,1.66,0.021
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,437.801639,16.352107,17.957779,17.605263,7.669887,453.846170,-0.53,-0.35,-0.57,0.021


### Feature-set 8: Exracting external factors using Fred API

In [40]:
from fredapi import Fred

fred = Fred(api_key="dfd5e2b3a7c9a714430cdf450699e698")

In [41]:
# Japanese Yen to U.S. Dollar Spot Exchange Rate
# U.S. Dollars to Euro Spot Exchange Rate
# Coinbase Bitcoin
feat_list = ["SP500", "DEXJPUS", "DEXUSEU", "CBBTCUSD"]
feat_df = pd.DataFrame()
for feat in feat_list:
    feature = fred.get_series(feat, "2020-10-31", "2023-10-31")
    feature = feature.to_frame(feat)
    feature.dropna(inplace=True)
    feat_df = pd.concat([feat_df, feature], axis=1)
feat_df.dropna(inplace=True)
feat_df

,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
2020-11-02,3310.24,104.80,1.1634,13573.93
2020-11-03,3369.16,104.50,1.1724,14050.59
2020-11-04,3443.44,104.39,1.1730,14160.00
2020-11-05,3510.45,103.67,1.1809,15636.60
2020-11-06,3509.44,103.32,1.1886,15596.33
...,...,...,...,...
2023-10-25,4186.77,149.94,1.0583,34489.91
2023-10-26,4137.23,150.44,1.0532,34140.00
2023-10-27,4117.37,149.60,1.0592,33905.16
2023-10-30,4166.82,149.01,1.0620,34485.49


In [42]:
NVDA_STOCK = pd.concat([NVDA_STOCK, feat_df], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volatility_UlcerIndex,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF,SP500,DEXJPUS,DEXUSEU,CBBTCUSD
2021-10-20,223.050003,224.330002,219.820007,221.029999,220.698547,14627600.0,-1.867203,-0.008425,221.726669,-0.282228,...,7.094438,217.316335,0.34,-0.02,1.19,0.000,4536.19,114.24,1.1643,66005.17
2021-10-21,220.970001,227.110001,220.830002,226.919998,226.579727,18759000.0,5.881180,0.026299,224.953334,1.444757,...,7.085631,217.404334,0.37,0.21,-0.95,0.000,4549.78,113.75,1.1643,62152.09
2021-10-22,228.229996,231.300003,225.610001,227.259995,226.919205,24938400.0,0.339478,0.001497,228.056666,1.370116,...,7.046261,217.932500,-0.25,-0.24,1.01,0.000,4544.90,113.54,1.1632,60671.75
2021-10-25,229.729996,233.550003,227.699997,231.660004,231.312622,23023500.0,4.393417,0.019176,230.970001,1.269370,...,6.950452,218.739167,0.58,0.49,-0.15,0.000,4566.48,113.70,1.1609,63102.83
2021-10-26,239.889999,252.589996,239.240005,247.169998,246.799347,48589800.0,15.486725,0.064806,246.333333,6.439780,...,6.626018,220.877833,0.04,-0.70,-0.31,0.000,4574.79,114.19,1.1590,60341.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,7.275335,455.716169,-1.58,-0.30,0.84,0.021,4186.77,149.94,1.0583,34489.91
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,7.403454,455.207336,-1.15,0.85,1.66,0.021,4137.23,150.44,1.0532,34140.00
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,7.669887,453.846170,-0.53,-0.35,-0.57,0.021,4117.37,149.60,1.0592,33905.16
2023-10-30,410.869995,417.660004,404.809998,411.609985,411.573792,38802800.0,6.609406,0.016189,411.359996,1.376439,...,7.962946,452.096670,1.15,-0.39,0.28,0.021,4166.82,149.01,1.0620,34485.49


### Feature-set 8: ADS features

In [43]:
ads = pd.read_excel("ads_index_most_current_vintage.xlsx")
ads.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
ads["Date"] = pd.to_datetime(ads["Date"], format="%Y:%m:%d")
ads = ads[(ads["Date"] >= "2020-10-31") & (ads["Date"] <= "2023-10-31")]
ads = ads.set_index("Date")
ads

,ADS_Index
Date,
2020-10-31,0.534722
2020-11-01,0.505453
2020-11-02,0.475630
2020-11-03,0.445241
2020-11-04,0.414276
...,...
2023-10-27,-0.511475
2023-10-28,-0.499639
2023-10-29,-0.486013


In [44]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2020-10-31 to 2023-10-31
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ADS_Index  1096 non-null   float64
dtypes: float64(1)
memory usage: 17.1 KB


In [45]:
NVDA_STOCK = pd.concat([NVDA_STOCK, ads], axis=1)
NVDA_STOCK.dropna(inplace=True)
NVDA_STOCK

,Open,High,Low,Close,Adj Close,Volume,Returns,Daily_Return,Typical_Price,Typical_Price_Return,...,Volatility_KeltnerChannel,Mkt-RF,SMB,HML,RF,SP500,DEXJPUS,DEXUSEU,CBBTCUSD,ADS_Index
2021-10-20,223.050003,224.330002,219.820007,221.029999,220.698547,14627600.0,-1.867203,-0.008425,221.726669,-0.282228,...,217.316335,0.34,-0.02,1.19,0.000,4536.19,114.24,1.1643,66005.17,1.019300
2021-10-21,220.970001,227.110001,220.830002,226.919998,226.579727,18759000.0,5.881180,0.026299,224.953334,1.444757,...,217.404334,0.37,0.21,-0.95,0.000,4549.78,113.75,1.1643,62152.09,1.032460
2021-10-22,228.229996,231.300003,225.610001,227.259995,226.919205,24938400.0,0.339478,0.001497,228.056666,1.370116,...,217.932500,-0.25,-0.24,1.01,0.000,4544.90,113.54,1.1632,60671.75,1.043380
2021-10-25,229.729996,233.550003,227.699997,231.660004,231.312622,23023500.0,4.393417,0.019176,230.970001,1.269370,...,218.739167,0.58,0.49,-0.15,0.000,4566.48,113.70,1.1609,63102.83,1.063390
2021-10-26,239.889999,252.589996,239.240005,247.169998,246.799347,48589800.0,15.486725,0.064806,246.333333,6.439780,...,220.877833,0.04,-0.70,-0.31,0.000,4574.79,114.19,1.1590,60341.21,1.066640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,433.980011,436.500000,415.549988,417.790009,417.753265,39837900.0,-18.838348,-0.044107,423.279999,-2.386560,...,455.716169,-1.58,-0.30,0.84,0.021,4186.77,149.94,1.0583,34489.91,-0.529790
2023-10-26,418.529999,422.559998,398.799988,403.260010,403.224548,54100100.0,-14.528717,-0.035397,408.206665,-3.626032,...,455.207336,-1.15,0.85,1.66,0.021,4137.23,150.44,1.0532,34140.00,-0.521524
2023-10-27,411.299988,412.059998,400.149994,405.000000,404.964386,41678400.0,1.739838,0.004306,405.736664,-0.606924,...,453.846170,-0.53,-0.35,-0.57,0.021,4117.37,149.60,1.0592,33905.16,-0.511475
2023-10-30,410.869995,417.660004,404.809998,411.609985,411.573792,38802800.0,6.609406,0.016189,411.359996,1.376439,...,452.096670,1.15,-0.39,0.28,0.021,4166.82,149.01,1.0620,34485.49,-0.471021


Saving the final dataframe as the *feature mart.*

In [46]:
# timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# filename = f'NVDA_feature_mart_{timestamp}.csv'
filename = f"NVDA_feature_mart.csv"
NVDA_STOCK.to_csv(filename, index=True)